In [105]:
from bs4 import BeautifulSoup as bs
import requests
from IPython.display import HTML
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy



## LEXICOGRAPHY : "##" Multiline Comment "#" Single line comment "###" Code that has been commented.



                     
                     PART I - Generating the fixed params for the HTTP GET request


In [5]:
## Firstly we need to download the data using Requests and BeautifulSoup.
## The basic URL used is http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247

# query_form is the form used to download the spreadsheets

query_form = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247')


In [6]:
# Creating a parse tree rooted at the BeautifulSoup object using a html parser.
soup = bs(query_form.text, "html.parser")

In [7]:
## Firstly, let us look at the form used to download the spreadsheets
HTML(query_form.text)

In [8]:
## It always helps to see the html source to decide what to do next.

###print(soup.prettify())

In [9]:
## We would like to automate the request, to do so we first look at how many inputs are required:

inputs = soup.find_all('input')
print ("There are", len(inputs) ,"inputs in all : \n")

for inp in inputs:
    print(inp['name'],"===>",inp['value'])
print("\n")

## Out of these inputs, there are some fields to select.

selections = soup.find_all('select')
print ("There are", len(selections) ,"selections in all : \n")

for sel in selections:
    print(sel["name"]) 

There are 10 inputs in all : 

ww_b_list ===> 1
ww_i_reportmodel ===> 133685247
ww_c_langue ===> 
ww_i_reportModelXsl ===> 133685270
ww_i_reportModelXsl ===> 133685271
zz_x_UNITE_ACAD ===> 
zz_x_PERIODE_ACAD ===> 
zz_x_PERIODE_PEDAGO ===> 
zz_x_HIVERETE ===> 
dummy ===> ok


There are 4 selections in all : 

ww_x_UNITE_ACAD
ww_x_PERIODE_ACAD
ww_x_PERIODE_PEDAGO
ww_x_HIVERETE


In [10]:
## We start creating the filters to go with our HTTP request
## The ww_x_GPS is set to -1 to get "Tous" c.f. slack. 

payload_head = {'ww_x_GPS' : '-1'}

## Observe that the name ww_i_reportModelXls appears twice and only one value interstes us.
#  We mean to download xls files so we LET the value for ww_i_reportModelXsl be overwritten!!

for inp in inputs[:5]:
    payload_head[inp['name']] = inp['value'] 
    

In [11]:
## Each selection has different options to choose from; We print them out here
soup.find_all('option')

[<option value="null"></option>,
 <option value="942293">Architecture</option>,
 <option value="246696">Chimie et génie chimique</option>,
 <option value="943282">Cours de mathématiques spéciales</option>,
 <option value="637841336">EME (EPFL Middle East)</option>,
 <option value="942623">Génie civil</option>,
 <option value="944263">Génie mécanique</option>,
 <option value="943936">Génie électrique et électronique </option>,
 <option value="2054839157">Humanités digitales</option>,
 <option value="249847">Informatique</option>,
 <option value="120623110">Ingénierie financière</option>,
 <option value="946882">Management de la technologie</option>,
 <option value="944590">Mathématiques</option>,
 <option value="945244">Microtechnique</option>,
 <option value="945571">Physique</option>,
 <option value="944917">Science et génie des matériaux</option>,
 <option value="942953">Sciences et ingénierie de l'environnement</option>,
 <option value="945901">Sciences et technologies du vivant</op

In [12]:
## We now concentrate on the section Informatique and add it to payload
info = soup.find('option', text = "Informatique")
payload_head[info.parent['name']] = info['value']

payload_head

{'ww_b_list': '1',
 'ww_c_langue': '',
 'ww_i_reportModelXsl': '133685271',
 'ww_i_reportmodel': '133685247',
 'ww_x_GPS': '-1',
 'ww_x_UNITE_ACAD': '249847'}

                END OF PART 1, WE HAVE THE BASIC ELEMENTS OF THE GET REQUEST READY.


In [13]:
## We are now capable of generating a request for any xls file.
## For this we make a few arrays to iterate over like year, semester etc.

years = ["2007-2008","2008-2009","2009-2010","2010-2011","2011-2012","2012-2013","2013-2014","2014-2015","2015-2016","2016-2017"]


payload_tail = {}

In [14]:
## Now, the fun part, we iterate for each year and get all the data for odd(autumn) semesters first
## and then the even(spring) semesters. Append the text vales of returned HTTP response to teh files
## array and their respective URLs to the urls array(Just in case).
## At each iteration, we upload the payload_head to send the appropriate HTTP GET request.

files = []

for y in years:
    year = soup.find('option', text = y)
    payload_tail[year.parent['name']] = year['value']

    one = soup.find('option', text = "Bachelor semestre 1")
    payload_tail[one.parent['name']] = one['value']
    payload_head.update(payload_tail)
    r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS?ww_i_reportModel=133685247', params = payload_head ) 
    files.append(r.text)
        
    six = soup.find('option', text = "Bachelor semestre 6")
    payload_tail[six.parent['name']] = six['value']
    payload_head.update(payload_tail)
    r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS?ww_i_reportModel=133685247', params = payload_head ) 
    files.append(r.text)
        
print("Downloaded", len(files) , "Files for Bachelors Sem 1 and Sem 6 from 2007 to 2016") 

Downloaded 20 Files for Bachelors Sem 1 and Sem 6 from 2007 to 2016


In [15]:
## Proof that the above code is not voodoo. :)
HTML(files[1])

### QUESTION ONE


In [16]:
file = pd.read_html(files[0])[0]
file.head(50)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Ces listes d'étudiants ne seront définitives q...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Informatique, 2007-2008, Bachelor semestre 1 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filičre opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,NaN,NaN
3,Monsieur,Arévalo Christian,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,169569,NaN,NaN
4,Monsieur,Aubelle Flavien,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,174905,NaN,NaN
5,Monsieur,Badoud Morgan,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,173922,NaN,NaN
6,Monsieur,Baeriswyl Jonathan,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,179406,NaN,NaN
7,Monsieur,Barroco Michael,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,179428,NaN,NaN
8,Monsieur,Belfis Nicolas,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,179324,NaN,NaN
9,Monsieur,Beliaev Stanislav,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,174597,NaN,NaN


In [60]:
## Creating a useable dataframe for the above html-based datadframe. 

def prepare_dataframe(f):
    file = pd.read_html(f)[0]
    period = file.loc[1, 0].split(',')
    file.drop([0,1], inplace = True)
    file.drop([2,3,4,5,6,7,8,9,11,12], axis = 1, inplace = True)
    file.columns = file.loc[2]
    file.columns.name = 0
    file["Sem " + period[2].split(' ')[3]] = int(period[1].split('-')[0])
    file.drop(2, inplace = True)
    file = file.set_index('No Sciper')
    return file

In [61]:
## Dreating an array with all the cleaned dataframes/ files
dfiles = []
for f in files:
    dfiles.append(prepare_dataframe(f))

In [62]:
## Concatenating all dataframes into one (indices not unique)
cfile = pd.concat(dfiles)
cfile.sort()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  app.launch_new_instance()


,Civilité,Nom Prénom,Sem 1,Sem 6
No Sciper,,,,
147008,Monsieur,Good Xavier,NaN,2010.0
147008,Monsieur,Good Xavier,2008.0,NaN
153762,Monsieur,Conus Johann,NaN,2007.0
159516,Monsieur,Raja Yanick,NaN,2007.0
159998,Madame,Jesse Julia,NaN,2007.0
161091,Madame,Grivet Ekaterina,NaN,2007.0
161127,Monsieur,Sahy François,NaN,2007.0
161212,Monsieur,Tamesna Rachid,2009.0,NaN
161220,Monsieur,Tourino Pablo,NaN,2007.0


In [64]:
## Collecting non null values for sem 6 and making a new data frame
## Dropping the second time someone does the Bachelo semester 1
## Keeping the last time someone does the Bachelor semester 6
sem6 = pd.DataFrame(cfile['Sem 6'].dropna()).reset_index().drop_duplicates('No Sciper', keep = 'last')
sem1 = pd.DataFrame(cfile['Sem 1'].dropna()).reset_index().drop_duplicates('No Sciper', keep = 'first')
gender = pd.DataFrame(cfile['Civilité']).reset_index()

In [65]:
final = pd.merge(sem1, sem6, on = 'No Sciper', how = 'inner')
final = pd.merge(final, gender, on ='No Sciper', how = 'inner')
final = final.set_index('No Sciper')

## Assumption : the question reads time between bachelor semster 1 and 6 so we consider only these two. 
## Time is the difference between the first time a student is in bachelor semester 1 to the last time 
## he/she is in semester 6

final["Time"] = (final["Sem 6"] - final["Sem 1"] + 0.5)*12

print( "population mean =", [final["Time"].mean()],"months")

## Printing the dataframe with the years and gender indexed on Sciper
final


population mean = [38.08641975308642] months


,Sem 1,Sem 6,Civilité,Time
No Sciper,,,,
169569,2007.0,2009.0,Monsieur,30.0
169569,2007.0,2009.0,Monsieur,30.0
174905,2007.0,2011.0,Monsieur,54.0
174905,2007.0,2011.0,Monsieur,54.0
174905,2007.0,2011.0,Monsieur,54.0
179406,2007.0,2010.0,Monsieur,42.0
179406,2007.0,2010.0,Monsieur,42.0
179406,2007.0,2010.0,Monsieur,42.0
179428,2007.0,2010.0,Monsieur,42.0


In [69]:
## Grouping men and women into the grouped groupby object
grouped = final.groupby('Civilité')

## Means per gender group
group_counts = grouped.mean()
group_counts.drop(['Sem 1', 'Sem 6'] , axis = 1)

,Time
Civilité,
Madame,35.194030
Monsieur,38.300552


In [70]:
## Statistical test: the Two-sample T-Test
## With the data we have we can reject the hypothesis that men and women take the same amount of time.

In [68]:
hommes = final.loc[final.Civilité == 'Monsieur']["Time"].values 
femmes = final.loc[final.Civilité == 'Madame']["Time"].values

stats.ttest_ind(a= hommes, b= femmes, equal_var=False)

Ttest_indResult(statistic=3.2489769011682865, pvalue=0.0016476806088724187)

### QUESTION TWO


In [71]:
##Preparing master dataframe similarly to the bachelor

master_files = []
semesters = [" 1" , " 2", " 3"]
for y in years:
    year = soup.find('option', text = y)
    payload_tail[year.parent['name']] = year['value']
    for sem in semesters:
        one = soup.find('option', text = "Master semestre" + sem)
        payload_tail[one.parent['name']] = one['value']
        payload_head.update(payload_tail)
        r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS?ww_i_reportModel=133685247', params = payload_head ) 
        master_files.append(r.text)
    
        

In [72]:
HTML(master_files[1])

In [73]:
def prepare_master_dataframe(f):
    file = pd.read_html(f)[0]
    period = file.loc[1, 0].split(',')
    file.drop([0,1], inplace = True)
    file.drop([2,3,5,7,8,9,11,12], axis = 1, inplace = True)
    file.columns = file.loc[2]
    file.columns.name = 0
    file["Sem " + period[2].split(' ')[3]] = int(period[1].split('-')[0])
    file.drop(2, inplace = True)
    file = file.set_index('No Sciper')
    return file

In [74]:
dfiles_master = []
for f in master_files:
    dfiles_master.append(prepare_master_dataframe(f))

In [45]:
dfiles_master[1]

,Civilité,Nom Prénom,Spécialisation,Mineur,Sem 2
No Sciper,,,,,
153066,Monsieur,Aeberhard François-Xavier,NaN,NaN,2007
180027,Madame,Agarwal Megha,NaN,NaN,2007
152232,Monsieur,Anagnostaras David,NaN,"Mineur en Management, technologie et entrepren...",2007
177395,Monsieur,Auroux Damien,Internet computing,NaN,2007
161970,Monsieur,Awalebo Joseph,NaN,NaN,2007
166258,Monsieur,Balet Ken,NaN,NaN,2007
173600,Monsieur,Barazzutti Raphaël Pierre,NaN,NaN,2007
178879,Monsieur,Bayramoglu Ersoy,NaN,NaN,2007
160213,Monsieur,Bécholey Alexandre,NaN,NaN,2007


In [75]:
cfile_master = pd.concat(dfiles_master)
cfile_master.sort()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  from ipykernel import kernelapp as app


,Civilité,Mineur,Nom Prénom,Sem 1,Sem 2,Sem 3,Spécialisation
No Sciper,,,,,,,
128911,Monsieur,NaN,Gulati Asheesh,NaN,NaN,2007.0,Internet computing
128911,Monsieur,NaN,Gulati Asheesh,NaN,2007.0,NaN,Internet computing
129093,Monsieur,NaN,Zhou Maoan,NaN,NaN,2007.0,NaN
129093,Monsieur,NaN,Zhou Maoan,NaN,2007.0,NaN,NaN
129326,Monsieur,NaN,Ni Zhong Zhong,NaN,2007.0,NaN,NaN
129326,Monsieur,NaN,Ni Zhong Zhong,NaN,NaN,2007.0,NaN
138088,Monsieur,NaN,Droz-dit-Busset Arnault,2007.0,NaN,NaN,NaN
145546,Monsieur,NaN,Clivaz Jean-Philippe,NaN,NaN,2007.0,NaN
145957,Monsieur,NaN,Hügli Michael,NaN,NaN,2007.0,NaN


In [52]:
cfile_master


,Civilité,Mineur,Nom Prénom,Sem 1,Sem 2,Sem 3,Spécialisation
No Sciper,,,,,,,
153066,Monsieur,NaN,Aeberhard François-Xavier,2007.0,NaN,NaN,NaN
180027,Madame,NaN,Agarwal Megha,2007.0,NaN,NaN,NaN
152232,Monsieur,NaN,Anagnostaras David,2007.0,NaN,NaN,NaN
177395,Monsieur,NaN,Auroux Damien,2007.0,NaN,NaN,NaN
161970,Monsieur,NaN,Awalebo Joseph,2007.0,NaN,NaN,NaN
166258,Monsieur,NaN,Balet Ken,2007.0,NaN,NaN,NaN
173600,Monsieur,NaN,Barazzutti Raphaël Pierre,2007.0,NaN,NaN,NaN
178879,Monsieur,NaN,Bayramoglu Ersoy,2007.0,NaN,NaN,NaN
154573,Madame,NaN,Benabdallah Zeineb,2007.0,NaN,NaN,NaN


In [96]:
msem2 = pd.DataFrame(cfile_master['Sem 2'].dropna()).reset_index().drop_duplicates('No Sciper', keep = 'last')
msem1 = pd.DataFrame(cfile_master['Sem 1'].dropna()).reset_index().drop_duplicates('No Sciper', keep = 'first')
msem3 = pd.DataFrame(cfile_master['Sem 3'].dropna()).reset_index().drop_duplicates('No Sciper', keep = 'last')

mgender = pd.DataFrame(cfile_master['Civilité']).reset_index().drop_duplicates('No Sciper', keep = 'first')
mineur = pd.DataFrame(cfile_master['Mineur'].dropna()).reset_index().drop_duplicates('No Sciper', keep = 'first')
spec = pd.DataFrame(cfile_master['Spécialisation'].dropna()).reset_index().drop_duplicates('No Sciper', keep = 'first')

In [85]:
final_master_1 = pd.merge(msem1, msem2, on = 'No Sciper', how = 'inner')

final_master_1 = pd.merge(final_master_1, mgender, on ='No Sciper', how = 'inner')
final_master_1 = final_master_1.set_index('No Sciper')

## Assumption : the question reads time between master semster 1 and 2 so we consider only these two. 
## Time is the difference between the first time a student is in master semester 1 to the last time 
## in master semster 2 but people can start in master sem 2, so we take the absolute value. 

final_master_1["Time"] = abs((final_master_1["Sem 2"] - final_master_1["Sem 1"] + 0.5)*12)

print( "population mean =", [final_master_1["Time"].mean()],"months")

## Printing the dataframe with the years and gender indexed on Sciper
final_master_1

population mean = [9.785340314136125] months


,Sem 1,Sem 2,Civilité,Time
No Sciper,,,,
153066,2007.0,2009.0,Monsieur,30.0
180027,2007.0,2007.0,Madame,6.0
152232,2007.0,2007.0,Monsieur,6.0
177395,2007.0,2007.0,Monsieur,6.0
161970,2007.0,2007.0,Monsieur,6.0
166258,2007.0,2007.0,Monsieur,6.0
173600,2007.0,2007.0,Monsieur,6.0
178879,2007.0,2007.0,Monsieur,6.0
154573,2007.0,2008.0,Madame,18.0


In [98]:
### People doing a Minor

final_master_2 = pd.merge(msem1, msem3, on = 'No Sciper', how = 'inner')
final_master_2 = pd.merge(final_master_2, mgender, on ='No Sciper', how = 'inner')
final_master_2 = pd.merge(final_master_2, mineur, on ='No Sciper', how = 'inner')
final_master_2 = final_master_2.set_index('No Sciper')

## Assumption : the question reads time between master semster 1 and 3 for those people with minors 
## so we consider only these three. 
## Time is the difference between the first time a student is in master semester 1 to the last time 
## in master semster 2 but people can start in master sem 2, so we take the absolute value. 

final_master_2["Time"] = abs((final_master_2["Sem 3"] - final_master_2["Sem 1"] + 0.5)*12)

mean_2 = final_master_2["Time"].mean()
print( "population mean =", [final_master_2["Time"].mean()],"months")

## Printing the dataframe with the years and gender indexed on Sciper
final_master_2


population mean = [20.72] months


,Sem 1,Sem 3,Civilité,Mineur,Time
No Sciper,,,,,
152232,2007.0,2008.0,Monsieur,"Mineur en Management, technologie et entrepren...",18.0
170451,2008.0,2010.0,Monsieur,Mineur en Etudes asiatiques contemporaines,30.0
166751,2008.0,2009.0,Monsieur,"Mineur en Management, technologie et entrepren...",18.0
167045,2008.0,2010.0,Monsieur,"Mineur en Management, technologie et entrepren...",30.0
170220,2009.0,2011.0,Monsieur,"Mineur en Management, technologie et entrepren...",30.0
170225,2009.0,2010.0,Monsieur,Mineur en Biocomputing,18.0
191589,2009.0,2010.0,Madame,"Mineur en Management, technologie et entrepren...",18.0
176564,2009.0,2010.0,Monsieur,Mineur en Biocomputing,18.0
171073,2009.0,2010.0,Monsieur,"Mineur en Management, technologie et entrepren...",18.0


In [113]:
### People doing a specialisation

final_master_3 = pd.merge(msem1, msem3, on = 'No Sciper', how = 'inner')
final_master_3 = pd.merge(final_master_3, mgender, on ='No Sciper', how = 'inner')
final_master_3 = pd.merge(final_master_3, spec, on ='No Sciper', how = 'inner')
final_master_3 = final_master_3.set_index('No Sciper')

## Assumption : the question reads time between master semster 1 and 3 for those people with minors 
## so we consider only these three. 
## Time is the difference between the first time a student is in master semester 1 to the last time 
## in master semster 2 but people can start in master sem 2, so we take the absolute value. 

final_master_3["Time"] = abs((final_master_3["Sem 3"] - final_master_3["Sem 1"] + 0.5)*12)

mean_3 = final_master_3["Time"].mean()
print( "population mean =", [final_master_3["Time"].mean()],"months")

## Printing the dataframe with the years and gender indexed on Sciper
final_master_3

population mean = [20.829694323144103] months


,Sem 1,Sem 3,Civilité,Spécialisation,Time
No Sciper,,,,,
153066,2007.0,2009.0,Monsieur,Internet computing,30.0
177395,2007.0,2008.0,Monsieur,Internet computing,18.0
166258,2007.0,2008.0,Monsieur,Internet computing,18.0
154573,2007.0,2009.0,Madame,Biocomputing,30.0
180072,2007.0,2008.0,Monsieur,Internet computing,18.0
160718,2007.0,2008.0,Monsieur,Foundations of Software,18.0
154080,2007.0,2009.0,Monsieur,Internet computing,30.0
161357,2007.0,2008.0,Monsieur,Foundations of Software,18.0
177399,2007.0,2008.0,Monsieur,Foundations of Software,18.0


In [108]:
## MEAN STAY AT EPFL MASTER
## The average time spent for finishing master 1 to master 3, WE IGNORE MASTER PROJECT AS IT IS NOT ALWAYS AT EPFL
## And the data is too sparse to be useful.

## The mean is simply the mean of means as mean is a linear operation 

mean_stay = numpy.mean([mean_2 , mean_3])
print("The mean stay for STUDIES in EPFL is " ,mean_stay)

The mean stay for STUDIES in EPFL is  20.7748471616


In [110]:
## MEAN PER SPECIALISATION
## Grouping specialisations into the grouped groupby object
grouped_spec = final_master_3.groupby('Spécialisation')

## Means per gender group
group_spec_counts = grouped_spec.mean()
group_spec_counts.drop(['Sem 1', 'Sem 3'] , axis = 1)



,Time
Spécialisation,
Biocomputing,24.000000
Computer Engineering - SP,19.894737
Computer Science Theory,18.000000
Data Analytics,18.000000
Foundations of Software,21.103448
Information Security - SP,18.000000
Internet Information Systems,18.000000
Internet computing,20.682353
Service science,20.400000


In [118]:
## COMPARING SAMPLE MEANS WITH THE POPULATION MEAN 
biocomputing = final_master_3.loc[final_master_3.Spécialisation == 'Biocomputing']["Time"].values 
print("Biocomputing", stats.ttest_1samp(a = biocomputing , popmean = mean_stay ))

computer_engg = final_master_3.loc[final_master_3.Spécialisation == 'Computer Engineering - SP']["Time"].values 
print("Computer Engineering", stats.ttest_1samp(a = computer_engg , popmean = mean_stay ))

computer_theory = final_master_3.loc[final_master_3.Spécialisation == 'Computer Science Theory']["Time"].values
print("Computer Science Theory", stats.ttest_1samp(a = computer_theory , popmean = mean_stay ))

data_analytics = final_master_3.loc[final_master_3.Spécialisation == 'Data Analytics']["Time"].values
print("Data Analytics", stats.ttest_1samp(a = data_analytics  , popmean = mean_stay ))
 


Biocomputing Ttest_1sampResult(statistic=0.93102142972203095, pvalue=0.42051903711803867)
Computer Engineering Ttest_1sampResult(statistic=-0.85334589168706076, pvalue=0.40468064365519851)
Computer Science Theory Ttest_1sampResult(statistic=nan, pvalue=nan)
Data Analytics Ttest_1sampResult(statistic=-inf, pvalue=0.0)


//anaconda/lib/python3.5/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


### QUESTION BONUS


In [127]:
final_master_4 = pd.merge(msem1, msem3, on = 'No Sciper', how = 'inner')
final_master_4 = pd.merge(final_master_4, mgender, on ='No Sciper', how = 'inner')
final_master_4 = final_master_4.set_index('No Sciper')

## MEAN PER GENDER
## Grouping specialisations into the grouped groupby object
grouped_gen = final_master_4.groupby('Civilité')

final_master_4["Time"] = abs((final_master_4["Sem 3"] - final_master_4["Sem 1"] + 0.5)*12)

## Means per gender group
group_gen_counts = grouped_gen.mean()
group_gen_counts.drop(['Sem 1', 'Sem 3'] , axis = 1)


,Time
Civilité,
Madame,19.783784
Monsieur,19.693252
